In [1]:
pip install SQLAlchemy==1.4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.17-cp310-cp310-linux_x86_64.whl size=1519930 sha256=73ecc0c1ed35d67bddd9a60422e9a941f6c54cf627946450a69f1226e77304e8
  Stored in directory: /root/.cache/pip/wheels/eb/d0/1e/ab0fcbac6f74a69b788a37fccd365fc67535b5ab79f97bd860
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.17 which is incompatible.


In [2]:
pip install pymssql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 18.0 MB/s eta 0:00:00


In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.0 MB/s eta 0:00:00


In [123]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import numpy as np
import warnings

import sklearn
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as rf
from catboost import CatBoostRegressor as sbr

In [5]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912"
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [6]:
df = pd.read_sql(
    """
    select *
    from data
    order by date, ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index(["date", "ticker"])
features = ["bm", "mom12m", "roeq"]

In [10]:
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)
df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d:
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )
)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (297). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (296). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (291). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (295). n_quantiles is set to n_samples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (277). 

In [12]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes = pd.DataFrame(codes).reset_index()
codes.columns = ["siccd", "industry"]

df = df.reset_index().merge(codes, on="siccd")
df = df.set_index(["date", "ticker"])

features.append("industry")

In [93]:
model = sbr(max_depth=4, num_trees=500, cat_features=["industry"], random_state=42)

In [94]:
X = df.drop(columns=['ret', 'actual'])
y = df['ret']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Catboost Model Analysis

In [96]:
model.fit(X_train, y_train)

Learning rate set to 0.132997
0:	learn: 1.0758249	total: 47ms	remaining: 23.5s
1:	learn: 1.0755272	total: 77.4ms	remaining: 19.3s
2:	learn: 1.0752685	total: 115ms	remaining: 19.1s
3:	learn: 1.0749724	total: 151ms	remaining: 18.7s
4:	learn: 1.0746244	total: 189ms	remaining: 18.7s
5:	learn: 1.0744276	total: 216ms	remaining: 17.8s
6:	learn: 1.0741143	total: 245ms	remaining: 17.3s
7:	learn: 1.0739164	total: 275ms	remaining: 16.9s
8:	learn: 1.0735631	total: 307ms	remaining: 16.8s
9:	learn: 1.0733310	total: 343ms	remaining: 16.8s
10:	learn: 1.0731269	total: 379ms	remaining: 16.8s
11:	learn: 1.0729193	total: 416ms	remaining: 16.9s
12:	learn: 1.0726649	total: 462ms	remaining: 17.3s
13:	learn: 1.0723002	total: 523ms	remaining: 18.1s
14:	learn: 1.0720511	total: 578ms	remaining: 18.7s
15:	learn: 1.0718180	total: 640ms	remaining: 19.4s
16:	learn: 1.0716608	total: 709ms	remaining: 20.1s
17:	learn: 1.0713618	total: 775ms	remaining: 20.8s
18:	learn: 1.0711841	total: 849ms	remaining: 21.5s
19:	learn: 

In [97]:
y_pred = model.predict(X_test)
model.score(X_test, y_test)

-0.00797826986983119

In [98]:
train_cols = X.columns

In [99]:
val = pd.DataFrame({'feature_importance': model.get_feature_importance(),
                    'feature_names': X.columns}).sort_values(by=['feature_importance'], ascending=False)


In [100]:
val.head(10)

,feature_importance,feature_names
52,5.909958,mom12m
53,4.464805,mom1m
55,3.256510,mom6m
46,2.976284,indmom
54,2.836952,mom36m
91,2.783393,sfe
85,2.613880,rsup
35,2.424776,ear
22,2.272434,chmom
44,2.106057,idiovol


## Linear Regression Model Analysis

In [101]:
val[val['feature_names'] == 'industry']

,feature_importance,feature_names
104,0.736593,industry


In [88]:
_, tick = zip(*np.array(X_train.index))
len(np.unique(np.array(tick)))

1392

In [74]:
arr = np.array(X_train.index)

In [76]:
a, b= zip(*arr)

In [103]:
transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)

model1 = LinearRegression(fit_intercept=False)

pipe1 = make_pipeline(
    transform1,
    model1
)

In [104]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['industry'])])),
                ('linearregression', LinearRegression(fit_intercept=False))])

In [105]:
pipe1.score(X_test, y_test)

-0.0017937164593118116

In [133]:
coefficients = pipe1.named_steps['linearregression'].coef_
# Get feature names after one-hot encoding
feature_names_transformed = pipe1.named_steps['columntransformer'].get_feature_names_out()


In [134]:
import pandas as pd

# Create a DataFrame with feature names and their corresponding coefficients
feature_importance = pd.DataFrame({
    'Feature': feature_names_transformed,
    'Coefficient': coefficients
})
feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Absolute Coefficient', ascending=False)
feature_importance = feature_importance.drop(columns=['Absolute Coefficient'])


In [135]:
feature_importance.head(10)

,Feature,Coefficient
57,remainder__ill,3778.813609
74,remainder__orgcap,-3.721791
56,remainder__idiovol,-1.354717
63,remainder__maxret,0.476712
11,onehotencoder__industry_Utilities,-0.389706
89,remainder__rd_mve,0.383392
46,remainder__dy,0.264572
33,remainder__chinv,-0.220666
93,remainder__roaq,0.205354
65,remainder__mom1m,-0.178442


Interestingly, for this model the industries have a higher coefficient value, meaning it is very important